### Geometric Brownian motion of the stock evaluation is given by:-

### $ds_t = r*S_t*dt + \sigma*s_t*dZ_t$

### If we try to solve above equation using normal calculus, we will have different solution but the second component in above equation represent stochastic component meaning a brownian process (fluctuation around some mean and variance)

### variance of fluctuation is proportional to dt (st.dev is proportional to sqrt(dt) using iid assumption), thus it implies that we cannot ignore the second order derivative in the taylor series expansion which results in a significant dt component

### thus final solution to the equation is below

### $S_t = S_{t-\Delta t}*\exp((r - \sigma^2/2)*\Delta t + \sigma*\sqrt(\Delta t)*z_t)$

In [1]:
import math
import numpy as np
import pandas as pd
import scipy.stats as scs
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
def pdf_of_normal_dist(x, mu, sigma):
    normal_z = (x - mu) / sigma
    return np.exp(-0.5 * z ** 2) / math.sqrt(2 * math.pi * sigma ** 2)

In [29]:
def simulate_gbm(s0, time_horizon, rf, vol, st_date, end_date):
    """
    function returns stock evaluation across dates based on parameters given as an argument
    """
    np.random.seed(250000) # so that I can repeat the same simulation
    df = pd.date_range(start=st_date, end=end_date, freq="B")
    time_steps = len(df)
    dt = 1 / 252 
    discount_factor = math.exp(-rf*dt) # single day discount factor
    
    rand_normal_matrix_for_dz = np.random.standard_normal((time_steps, 1))
    
    stock = np.zeros_like(rand_normal_matrix_for_dz) # same size for number of days
    stock[0] = s0
    
    
    for t in range(1, time_steps):
        stock[t] = stock[t-1] * np.exp((rf - vol**2/2)*dt + vol*np.sqrt(dt)*rand_normal_matrix_for_dz[t])
    
    gbm = pd.DataFrame(stock, index=df, columns=["index"])
    
    return gbm

In [33]:
stock_profile = simulate_gbm(100, 10, 0.05, 0.2, "01-01-2012", "01-01-2022")

In [51]:
def calculate_historical_annualized_vol_of_lognormal_returns(gbm):
    gbm['returns'] = np.log(gbm['index'] / gbm['index'].shift(1))
    
    # variance (multiplied by 252 to annualized it)
    gbm['realized_var'] = 252 * np.cumsum(gbm.returns**2) / np.arange(len(gbm))
    gbm['realized_vol'] = np.sqrt(gbm['realized_var'])
    gbm = gbm.dropna()
    
    return gbm

In [52]:
vol_returns = calculate_historical_annualized_vol_of_lognormal_returns(stock_profile)

In [55]:
# this method is copied from book
def print_statistics(data):
    print("RETURN SAMPLE STATISTICS")
    print("---------------------------------------------")
    print("Mean of Daily  Log Returns %9.6f" % np.mean(data['returns']))
    print("Mean of Annua. Log Returns %9.6f" %
          (np.mean(data['returns']) * 252))
    print("Std  of Annua. Log Returns %9.6f" %
          (np.std(data['returns']) * math.sqrt(252)))
    print("---------------------------------------------")
    print("Skew of Sample Log Returns %9.6f" % scs.skew(data['returns']))
    print("Skew Normal Test p-value   %9.6f" %
          scs.skewtest(data['returns'])[1])
    print("---------------------------------------------")
    print("Kurt of Sample Log Returns %9.6f" % scs.kurtosis(data['returns']))
    print("Kurt Normal Test p-value   %9.6f" %
          scs.kurtosistest(data['returns'])[1])
    print("---------------------------------------------")
    print("Normal Test p-value        %9.6f" %
          scs.normaltest(data['returns'])[1])
    print("---------------------------------------------")
    print("Realized Volatility        %9.6f" % data['rea_vol'].iloc[-1])
    print("Realized Variance          %9.6f" % data['rea_var'].iloc[-1])
    
    return